# Use PyTorch to recognize hand-written digits with Watson Machine Learning REST API

This notebook contains steps and code to demonstrate support of PyTorch Deep Learning experiments in Watson Machine Learning Service. It introduces commands for getting data, training experiments, persisting pipelines, publishing models, deploying models and scoring.

Some familiarity with cURL is helpful. This notebook uses cURL examples.


## Learning goals

The learning goals of this notebook are:

-  Working with Watson Machine Learning experiments to train Deep Learning models.
-  Downloading computed models to local storage.
-  Online deployment and scoring of trained model.


## Contents

This notebook contains the following parts:

1.	[Setup](#setup)   
2.  [Model definition](#model_definition)  
3.	[Experiment Run](#run)  
4.	[Historical runs](#runs)  
5.	[Deploy and Score](#deploy_and_score)  
6.	[Cleaning](#cleaning)   
7.	[Summary and next steps](#summary)

<a id="setup"></a>
## 1. Set up the environment

Before you use the sample code in this notebook, you must perform the following setup tasks:

-  Contact with your Cloud Pack for Data administrator and ask him for your account credentials

### Connection to WML

Authenticate the Watson Machine Learning service on IBM Cloud Pack for Data. You need to provide platform `url`, your `username` and `password`.

In [ ]:
%env USERNAME=
%env PASSWORD=
%env DATAPLATFORM_URL=

%env SPACE_ID=

<a id="wml_token"></a>
### Getting WML authorization token for further cURL calls

<a href="https://cloud.ibm.com/docs/cloud-object-storage?topic=cloud-object-storage-curl#curl-token" target="_blank" rel="noopener no referrer">Example of cURL call to get WML token</a>

In [ ]:
%%bash --out token

curl -sk -X GET \
    --user $USERNAME:$PASSWORD \
    --header "Accept: application/json" \
    "$DATAPLATFORM_URL/v1/preauth/validateAuth" \
    | cut -d '"' -f 44

In [ ]:
%env TOKEN=$token 

<a id="space_creation"></a>
### Space creation
**Tip:** If you do not have `space` already created, please convert below three cells to `code` and run them.

First of all, you need to create a `space` that will be used in all of your further cURL calls. 
If you do not have `space` already created, below is the cURL call to create one.

<a href="https://cpd-spaces-api.eu-gb.cf.appdomain.cloud/#/Spaces/spaces_create" 
target="_blank" rel="noopener no referrer">Space creation</a>

Space creation is asynchronous. This means that you need to check space creation status after creation call.
Make sure that your newly created space is `active`.

<a href="https://cpd-spaces-api.eu-gb.cf.appdomain.cloud/#/Spaces/spaces_get" 
target="_blank" rel="noopener no referrer">Get space information</a>

<a id="model_definition"></a>
## 2. Model definition 

This section provides samples about how to store model definition via cURL calls.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Model%20Definitions/model_definitions_create" 
target="_blank" rel="noopener no referrer">Store a model definition for Deep Learning experiment</a>

In [68]:
%%bash --out model_definition_payload

MODEL_DEFINITION_PAYLOAD='{"name": "PyTorch Hand-written Digit Recognition", "space_id": "'"$SPACE_ID"'", "description": "PyTorch Hand-written Digit Recognition", "tags": ["DL", "PyTorch"], "version": "v1", "platform": {"name": "python", "versions": ["3.7"]}, "command": "pytorch_v_1.1_mnist_onnx.py --epochs 10 --debug-level debug"}'
echo $MODEL_DEFINITION_PAYLOAD | python -m json.tool

In [69]:
%env MODEL_DEFINITION_PAYLOAD=$model_definition_payload

env: MODEL_DEFINITION_PAYLOAD={
    "name": "PyTorch Hand-written Digit Recognition",
    "space_id": "059f6303-90c3-4732-b046-3f4ee43b3362",
    "description": "PyTorch Hand-written Digit Recognition",
    "tags": [
        "DL",
        "PyTorch"
    ],
    "version": "v1",
    "platform": {
        "name": "python",
        "versions": [
            "3.7"
        ]
    },
    "command": "pytorch_v_1.1_mnist_onnx.py --epochs 10 --debug-level debug"
}


In [ ]:
%%bash --out model_definition_id

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$MODEL_DEFINITION_PAYLOAD" \
    "$DATAPLATFORM_URL/ml/v4/model_definitions?version=2020-08-01"| grep '"id": ' | awk -F '"' '{ print $4 }'

In [ ]:
%env MODEL_DEFINITION_ID=$model_definition_id

<a id="model_preparation"></a>
### Model preparation

Download files with pytorch code. You can either download it via link below or run the cell below the link.

<a href="https://github.com/IBM/watson-machine-learning-samples/raw/master/cpd/definitions/pytorch/mnist/pytorch-onnx_v1_3.zip" 
target="_blank" rel="noopener no referrer">Download pytorch-model.zip</a>

In [ ]:
%%bash

wget https://github.com/IBM/watson-machine-learning-samples/raw/master/cpd3.5/definitions/pytorch/mnist/pytorch_onnx_v1_3.zip \
     -O pytorch-onnx_v1_3.zip

**Tip**: Convert below cell to code and run it to see model deinition's code.

<a id="def_upload"></a>
### Upload model for the model definition

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Model%20Definitions/model_definitions_upload_model" 
target="_blank" rel="noopener no referrer">Upload model for the model definition</a>

In [ ]:
%%bash

curl -sk -X PUT \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data-binary "@pytorch-onnx_v1_3.zip" \
    "$DATAPLATFORM_URL/ml/v4/model_definitions/$MODEL_DEFINITION_ID/model?version=2020-08-01&space_id=$SPACE_ID" \
    | python -m json.tool

<a id="run"></a>
## 3. Experiment run

This section provides samples about how to trigger Deep Learning experiment via cURL calls.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Trainings/trainings_create" 
target="_blank" rel="noopener no referrer">Schedule a training job for Deep Learning experiment</a>

Specify the source files folder where you have stored your training data. The path should point to a local repository on Watson Machine Learning Accelerator that your system administrator has set up for your use.

**Action:**
Change `training_data_references: location: path: ...`

In [70]:
%%bash --out training_payload

TRAINING_PAYLOAD='{"training_data_references": [{"name": "training_input_data", "type": "fs", "connection": {}, "location": {"path": "pytorch-mnist"}, "schema": {"id": "idmlp_schema", "fields": [{"name": "text", "type": "string"}]}}], "results_reference": {"name": "MNIST results", "connection": {}, "location": {"path": "spaces/'"$SPACE_ID"'/assets/experiment"}, "type": "fs"}, "tags": [{"value": "tags_pytorch", "description": "Tags PyTorch"}], "name": "PyTorch hand-written Digit Recognition", "description": "PyTorch hand-written Digit Recognition", "model_definition": {"id": "'"$MODEL_DEFINITION_ID"'", "command": "pytorch_v_1.1_mnist_onnx.py --epochs 10 --debug-level debug", "hardware_spec": {"name": "K80", "nodes": 1}, "software_spec": {"name": "pytorch-onnx_1.3-py3.7"}, "parameters": {"name": "PyTorch_mnist", "description": "PyTorch mnist recognition"}}, "space_id": "'"$SPACE_ID"'"}'
echo $TRAINING_PAYLOAD | python -m json.tool

In [71]:
%env TRAINING_PAYLOAD=$training_payload

env: TRAINING_PAYLOAD={
    "training_data_references": [
        {
            "name": "training_input_data",
            "type": "fs",
            "connection": {},
            "location": {
                "path": "pytorch-mnist"
            },
            "schema": {
                "id": "idmlp_schema",
                "fields": [
                    {
                        "name": "text",
                        "type": "string"
                    }
                ]
            }
        }
    ],
    "results_reference": {
        "name": "MNIST results",
        "connection": {},
        "location": {
            "path": "spaces/059f6303-90c3-4732-b046-3f4ee43b3362/assets/experiment"
        },
        "type": "fs"
    },
    "tags": [
        {
            "value": "tags_pytorch",
            "description": "Tags PyTorch"
        }
    ],
    "name": "PyTorch hand-written Digit Recognition",
    "description": "PyTorch hand-written Digit Recognition",
    "model_definition"

In [ ]:
%%bash --out training_id

curl -sk -X  POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$TRAINING_PAYLOAD" \
    "$DATAPLATFORM_URL/ml/v4/trainings?version=2020-08-01" | awk -F'"id":' '{print $2}' | cut -c2-37

In [ ]:
%env TRAINING_ID=$training_id

<a id="training_details"></a>
### Get training details
Treining is an asynchronous endpoint. In case you want to monitor training status and details,
you need to use a GET method and specify which training you want to monitor by usage of training ID.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Trainings/trainings_get" 
target="_blank" rel="noopener no referrer">Get information about training job</a>

### Get training status

In [ ]:
%%bash

STATUS=$(curl -sk -X GET\
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    "$DATAPLATFORM_URL/ml/v4/trainings/$TRAINING_ID?space_id=$SPACE_ID&version=2020-08-01")
    
STATUS=${STATUS#*state\":\"}
STATUS=${STATUS%%\"*}
echo $STATUS

Please make sure that training is completed before you go to the next sections.
Monitor `state` of your training by running above cell couple of times.

<a id="runs"></a>
## 4. Historical runs

In this section you will see cURL examples describing how to get historical training runs information.

Output should be similar to the output from training creation but you should see more trainings entries.  
Listing trainings:

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Trainings/trainings_list" 
target="_blank" rel="noopener no referrer">Get list of historical training jobs information</a>

In [ ]:
%%bash

HISTORICAL_TRAINING_LIMIT_TO_GET=2

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    "$DATAPLATFORM_URL/ml/v4/trainings?space_id=$SPACE_ID&version=2020-08-01&limit=$HISTORICAL_TRAINING_LIMIT_TO_GET" \
    | python -m json.tool

<a id="training_cancel"></a>
### Cancel training run

**Tip:** If you want to cancel your training, please convert below cell to `code`, specify training ID and run.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Trainings/trainings_delete" 
target="_blank" rel="noopener no referrer">Canceling training</a>

---

<a id="deploy_and_score"></a>
## 5. Deploy and Score

In this section you will learn how to deploy and score pipeline model as webservice using WML instance.

Before deployment creation, you need store your model in WML repository.
Please see below cURL call example how to do it.

Download `request.json` with repository request json for model storing.

In [ ]:
%%bash --out request_json

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    "$DATAPLATFORM_URL/v2/asset_files/experiment/$TRAINING_ID/assets/$TRAINING_ID/resources/wml_model/request.json?space_id=$SPACE_ID&version=2020-08-01" \
    | python -m json.tool

In [ ]:
%env MODEL_PAYLOAD=$request_json

<a id="model_store"></a>
### Store Deep Learning model

Store information about your model to WML repository.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Models/models_create" 
target="_blank" rel="noopener no referrer">Model storing</a>

In [ ]:
%%bash --out model_details

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$MODEL_PAYLOAD" \
    "$DATAPLATFORM_URL/ml/v4/models?version=2020-08-01&space_id=$SPACE_ID"

In [ ]:
%env MODEL_DETAILS=$model_details

In [ ]:
%%bash --out model_id

echo $MODEL_DETAILS | awk -F '"id": ' '{ print $5 }' | cut -d '"' -f 2

In [ ]:
%env MODEL_ID=$model_id

<a id="deployment_creation"></a>
### Deployment creation

An Deep Learning online deployment creation is presented below.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Deployments/deployments_create" 
target="_blank" rel="noopener no referrer">Create deployment</a>

In [ ]:
%%bash --out deployment_payload

DEPLOYMENT_PAYLOAD='{"space_id": "'"$SPACE_ID"'","name": "PyTorch Mnist deployment", "description": "PyTorch model to recognize hand-written digits","online": {},"hardware_spec": {"name": "S"},"asset": {"id": "'"$MODEL_ID"'"}}'
echo $DEPLOYMENT_PAYLOAD | python -m json.tool

In [ ]:
%env DEPLOYMENT_PAYLOAD=$deployment_payload

In [ ]:
%%bash

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$DEPLOYMENT_PAYLOAD" \
    "$DATAPLATFORM_URL/ml/v4/deployments?version=2020-08-01"

In [ ]:
%%bash --out deployment_id

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data "$DEPLOYMENT_PAYLOAD" \
    "$DATAPLATFORM_URL/ml/v4/deployments?version=2020-08-01" \
    | grep '"id": ' | awk -F '"' '{ print $4 }' | sed -n 3p

In [ ]:
%env DEPLOYMENT_ID=$deployment_id

<a id="deployment_details"></a>
### Get deployment details
As deployment API is asynchronous, please make sure your deployment is in `ready` state before going to the next points.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Deployments/deployments_get" 
target="_blank" rel="noopener no referrer">Get deployment details</a>

In [ ]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    "$DATAPLATFORM_URL/ml/v4/deployments/$DEPLOYMENT_ID?space_id=$SPACE_ID&version=2020-08-01" \
    | python -m json.tool

<a id="input_score"></a>
### Prepare scoring input data
**Hint:** You may need to install numpy using following command `!pip install numpy`

In [ ]:
!wget -q https://github.com/IBM/watson-machine-learning-samples/raw/master/cpd3.5/data/mnist/mnist.npz

In [ ]:
import numpy as np

mnist_dataset = np.load('mnist.npz')
test_mnist = mnist_dataset['x_test']

In [ ]:
image_1 = [test_mnist[0].tolist()]
image_2 = [test_mnist[1].tolist()]

<a id="webservice_score"></a>
### Scoring of a webservice
If you want to make a `score` call on your deployment, please follow a below method:

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Deployment%20Jobs/deployment_jobs_create" 
target="_blank" rel="noopener no referrer">Create deployment job</a>

In [ ]:
%%bash -s "$image_1" "$image_2"

curl -sk -X POST \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    --header "Accept: application/json" \
    --data '{"space_id": "$SPACE_ID","input_data": [{"values": ['"$1"', '"$2"']}]}' \
    "$DATAPLATFORM_URL/ml/v4/deployments/$DEPLOYMENT_ID/predictions?version=2020-08-01" \
    | python -m json.tool

<a id="deployments_list"></a>
### Listing all deployments

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Deployments/deployments_list" 
target="_blank" rel="noopener no referrer">List deployments details</a>

In [ ]:
%%bash

curl -sk -X GET \
    --header "Authorization: Bearer $TOKEN" \
    --header "Content-Type: application/json" \
    "$DATAPLATFORM_URL/ml/v4/deployments?space_id=$SPACE_ID&version=2020-08-01" \
    | python -m json.tool

<a id="cleaning"></a>
## 6. Cleaning section

Below section is useful when you want to clean all of your previous work within this notebook.
Just convert below cells into the `code` and run them.

<a id="training_delete"></a>
### Delete training run
**Tip:** You can completely delete a training run with its metadata.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Trainings/trainings_delete" 
target="_blank" rel="noopener no referrer">Deleting training</a>

<a id="deployment_delete"></a>
### Deleting deployment
**Tip:** You can delete existing deployment by calling DELETE method.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Deployments/deployments_delete" 
target="_blank" rel="noopener no referrer">Delete deployment</a>

<a id="model_delete"></a>
### Delete model from repository
**Tip:** If you want to completely remove your stored model and model metadata, just use a DELETE method.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Models/models_delete" 
target="_blank" rel="noopener no referrer">Delete model from repository</a>

<a id="def_delete"></a>
### Delete model definition
**Tip:** If you want to completely remove your model definition, just use a DELETE method.

<a href="https://watson-ml-v4-api.mybluemix.net/wml-restapi-cloud.html#/Model%20Definitions/model_definitions_delete" 
target="_blank" rel="noopener no referrer">Delete model definition</a>

<a id="summary"></a>
## 7. Summary and next steps

 You successfully completed this notebook!.
 
 You learned how to use `cURL` calls to store, deploy and score a PyTorch Deep Learning model in WML. 

### Authors

**Jan Sołtysik**, Intern in Watson Machine Learning at IBM

Copyright © 2020 IBM. This notebook and its source code are released under the terms of the MIT License.